In [2]:
#standard packages
import pandas as pd
import numpy as np
import math

# plotting
import matplotlib.pyplot as plt
%matplotlib inline
#from labellines import labelLine, labelLines

#administrative packages
import os
import operator
import datetime
mydate = datetime.datetime.now()
from datetime import date,timedelta
import time
start = time.time()

planner_initials = {'ABC': 'Alan Carreon',
                     'AS': 'Ana Spinola',
                     'AMS': 'Andrew Smith',
                     'CG': 'Chip Griffin',
                     'EB': 'Ethan T Bindernagel',
                     'GDK': 'Greg Kapovich',
                     'GV': 'Gerardo Victoria',
                     'HEC': 'Haley E Croffoot',
                     'HH': 'Haley Hubbard',
                     'JG': 'Jessica J Gonzalez',
                     'JCav': 'Jeanine Cavalli',
                     'KN': 'Ken Nodder',
                     'OA': 'Ozzy Arce',
                     'SKG': 'Simar Gill',
                     'SP': 'Sukhamrit S Purewal',
                     'TC': 'Trishia Caguiat'}
planner_names = {v:k for k,v in planner_initials.items()}
planner_names['Haley Hubbard'] = 'HEC'
planner_initials['HH'] = 'Haley E Croffoot'
Complete_Closed = ['Approved - Closed', 'Closed', 'Not Approved - Closed']
Planner_Closed = ['Close Out', 'Approved', 'Not Approved']
Ent = {
 'Zoning Amendment': 'Amend',
 'Use Permit Minor': 'MUP',
 'Design Review': 'DR',
 'Design Review Oversized Home': 'DR',
 'Design Review Other': 'DR',
 'Design Review Commercial': 'DR',
 'General Plan Amendment': 'Amend',
 'Design Review Antenna': 'DR',
 'Design Review Residential': 'DR',
 'Use Permit Conditional': 'CUP',
 'Use Permit Administrative': 'AUP',
 'ZCL':'ZCL',
 'Variance': 'Vari',
 'Rezoning': 'ReZone',
 'Tree Dripline Encroachment': 'Tree',
 'Tree Removal Permit': 'Tree',
 '': 'Other',
 'Other': 'Other',
 'Tentative Map Major Subdivision': 'Maj Sub',
 'Drip Line Encroachment': 'Other',
 'Tentative Map Minor Subdivision': 'Min Sub',
 'Tentative Map Condo Conversion': 'Conv',
 'Hillside Performance Standards': 'Other'}
Ent_names = {v:k for k,v in Ent.items()}
Ent_names['Other'] = 'Other'

path = r'C:\Users\nelms\Downloads\BlueTest_Engineering_SDP (3).csv'
SDP = pd.read_csv(path, skiprows=1)

date_fields = ['Open Date', 'Status Date']
for d in date_fields:
    SDP[d] = SDP[d].apply(lambda s: pd.to_datetime(s, infer_datetime_format=True))
    
SDP = SDP[[
    'Permit #',
    'Record Status',
    'Open Date',
    'Status Date',
    'Project Type',
    'UPDATED BY',
    'Description'
    ]].drop_duplicates(keep='last').set_index('Permit #')
SDP.loc[SDP['UPDATED BY']=='AA CONV', 'UPDATED BY'] = ''
sdp_t = {
    'Subdivision':'Subdivision', 
    'COMM':'Commercial',
    'RES':'Residential', 
    'Residential':'Residential',
    'Commercial':'Commercial'
    }
SDP['Project Type'] = SDP['Project Type'].map(sdp_t)

WS = SDP

def colnum_string(n):
    string = ""
    n = n + 1
    while n > 0:
        n, remainder = divmod(n - 1, 26)
        string = chr(65 + remainder) + string
    return string

def rng_create(A, one, B, two):
    return str(colnum_string(A)) + str(one) + ":" + str(colnum_string(B)) + str(two)

def get_col_widths(dataframe):
    # First we find the maximum length of the index column   
    idx_max = max([len(str(s)) for s in dataframe.index.values] + [len(str(dataframe.index.name))])
    # Then, we concatenate this to the max of the lengths of column name and its values for each column, left to right
    return [idx_max] + [max([len(str(s)) for s in dataframe[col].values] + [len(col)]) for col in dataframe.columns]

def normalize_types(lyr):
        str_list = list(lyr.select_dtypes(include=['datetime','period[Q-DEC]','O']))
        convert_dict = {s:str for s in str_list}
        no_str = [l for l in list(lyr) if l not in str_list]
        no_dict = lyr[no_str].dtypes.to_dict()
        convert_dict.update(no_dict)
        return lyr.astype(convert_dict)

new_file = 'SDP_Status.xlsx'
folder = r'O:\CDD\PLANNING\AN\Projects\Accela Reporting'

import xlsxwriter as xlsx
wrkbk = xlsx.Workbook(os.path.join(folder, new_file))

for tbl, name, caption in [(WS, 'Permits', 'SDPs with status in 2018 to 2020')]:
    worksheet = wrkbk.add_worksheet(name)
    
    lyr = tbl.copy()
    
    lyr = normalize_types(lyr)
    
    header = np.array([[lyr.index.name] + list(lyr)])
    data = lyr.reset_index().to_numpy()
    if data.shape[1] != header.shape[1]:        
        dif = data.shape[1] - header.shape[1]
        fix_head = [lyr.index.name] + list(lyr)
        for r in range(dif):
            fix_head = [' '] + fix_head
        header = np.array([fix_head])
    table = np.vstack((header,data))
    numRows,numColumns = table.shape
    
    cols = [{'header':h} for h in header[0]]
    
    rowoffset,coloffset = 1,0
    
    if len(caption) > 0:
        cell_format = wrkbk.add_format()
        cell_format.set_bold() 
        worksheet.merge_range('A1:B1', caption, cell_format)
        rowoffset += 1
    
    length,width = data.shape
    table_range = rng_create(0+coloffset, 0+rowoffset, width-1+coloffset, length+rowoffset)
    
    worksheet.add_table(table_range, {'data': data,
                                      'style': 'Table Style Light 1',
                                       'columns': cols
                                 })
    for e,c in enumerate(get_col_widths(lyr)):
        e = e+coloffset
        cw = colnum_string(e)
        cw = cw + ':' + cw
        if c > 30:
            c = 30
        worksheet.set_column(cw, c+1)
    
wrkbk.close()

In [70]:
## Location of CSV from Accela Ad Hoc
path = r'O:\CDD\PLANNING\AN\Projects\Accela Reporting\Building\Weekly_Status_BuildCheck.csv'
WS = pd.read_csv(path).set_index('RECORD ID')
WS.index.name = 'Record ID'

WS.rename(columns={
    'ACTION BY NAME FML#':'Intaker', 
    'ASSIGNED NAME FML#':'Engineer', 
    'RECORD OPEN DATE':'Open Date', 
    'DATE ASSIGNED':'Date Assigned',
    'DESCRIPTION':'Description'
}, inplace=True)
Admin_fields = ['Address', 'Description', 'Intaker']
Date_fields = ['Open Date', 'Date Assigned']
PCE_fields = ['Engineer']

WS.dropna(subset=['Intaker', 'Engineer'], inplace=True)
# ADMIN
# Initials
#WS['Intaker'] = WS['Intaker'].apply(lambda x: ''.join([n[0] for n in x.split()]))
# Last Name
WS['Intaker'] = WS['Intaker'].apply(lambda x: x.split()[-1])
# descript
def adds(add):
    add = add.replace("(N) ","").replace("T.I.","TI").strip()
    for sep in [";", ",", " (", "-", " &", " TO "]:
        if sep in add:
            add = add.split(sep)[0].strip()
    add = " ".join(add.split()).replace('"','')
    while len(add) > 30:
        for sep in [" AND "]:
            add = add.split(sep)[0].strip()
        add = " ".join(add.split()[:-1])
    return add
WS['Description'] = WS['Description'].apply(lambda x: adds(x))
# Address
WS['Address'] = WS['STREET NBR'].astype(str) + ' ' + WS['STREET NAME']
WS['Address'] = WS['Address'].apply(lambda x: str(x).title().strip())

# DATES
WS.loc[WS['Date Assigned'].isna(), 'Date Assigned'] = WS.loc[WS['Date Assigned'].isna(), 'DATE STATUS']
for d in Date_fields:
    WS[d] = WS[d].apply(lambda x: pd.to_datetime(x, format="%m/%d/%Y"))

WS = WS[Admin_fields + Date_fields + PCE_fields].sort_values(by=['Record ID'])

In [74]:
from datetime import datetime

def colnum_string(n):
    string = ""
    n = n + 1
    while n > 0:
        n, remainder = divmod(n - 1, 26)
        string = chr(65 + remainder) + string
    return string

def rng_create(A, one, B, two):
    return str(colnum_string(A)) + str(one) + ":" + str(colnum_string(B)) + str(two)

def get_col_widths(dataframe):
    # First we find the maximum length of the index column   
    idx_max = max([len(str(s)) for s in dataframe.index.values] + [len(str(dataframe.index.name))])
    # Then, we concatenate this to the max of the lengths of column name and its values for each column, left to right
    return [idx_max] + [max([len(str(s)) for s in dataframe[col].values] + [len(col)]) for col in dataframe.columns]

def normalize_types(lyr):
        str_list = list(lyr.select_dtypes(include=['datetime','period[Q-DEC]','O']))
        convert_dict = {s:str for s in str_list}
        no_str = [l for l in list(lyr) if l not in str_list]
        no_dict = lyr[no_str].dtypes.to_dict()
        convert_dict.update(no_dict)
        return lyr.astype(convert_dict)

new_file = 'PCE_Status.xlsx'
folder = r'O:\CDD\PLANNING\AN\Projects\Accela Reporting\Weekly_Building'

import xlsxwriter as xlsx
wrkbk = xlsx.Workbook(os.path.join(folder, new_file))
"""
for OG, name in [(WS, 'Permits')]:
    worksheet = wrkbk.add_worksheet(name)
    
    lyr = OG.copy()
    
    lyr = normalize_types(lyr)
    
    header = np.array([[lyr.index.name] + list(lyr)])
    data = lyr.reset_index().to_numpy()
    if data.shape[1] != header.shape[1]:        
        dif = data.shape[1] - header.shape[1]
        fix_head = [lyr.index.name] + list(lyr)
        for r in range(dif):
            fix_head = [' '] + fix_head
        header = np.array([fix_head])
    table = np.vstack((header,data))
    numRows,numColumns = table.shape
    
    idx = [0]
    if type(lyr.index) == pd.MultiIndex:
        idx.append(1)
        table[0,0:2] = list(lyr.index.names)
    else:
        table[0,0:1] = list(lyr.index.names)

    for c in range(numColumns):
        for r in range(numRows):
            format_dict = {'font_name':'Arial'}
            
            # TOP
            if (r == 0) & (c not in idx):
                format_dict['bottom'] = 2
                format_dict['bold'] = True
                format_dict['text_wrap'] = True
            # INDEX NAME
            if (r == 0) & (c in idx):
                format_dict['italic'] = True
            # RIGHT
            elif (r != 0) & (c == range(numColumns)[-1]):
                format_dict['right'] = 2
                if (r == range(numRows)[-1]):
                    format_dict['bottom'] = 2
            # LEFT
            elif (r != 0) & (c in idx):
                format_dict['bold'] = True
                if (1 not in idx)or(c == 1):
                    format_dict['right'] = 2
            # BOTTOM
            elif (r == range(numRows)[-1]) & (c not in idx):
                format_dict['bottom'] = 2
                if (c == range(numColumns)[-1]):
                    format_dict['right'] = 2
            item = table[r][c]
            
            cell_format = wrkbk.add_format(format_dict)
            worksheet.write(r, c, item, cell_format)
        if c not in idx:
            worksheet.set_column(c, c, 10)
        else:
            try:
                worksheet.set_column(c, c, len(max(table[:,c], key=len))+1)
            except:
                print(table[:,c])

"""

def get_date_range(series, form='%m/%d/%Y'):
    dates = [pd.to_datetime(s, infer_datetime_format=True) for s in series.values]
    max_d = max(dates).strftime(form)
    min_d = min(dates).strftime(form)
    return min_d, max_d
WSmn, WSmx = get_date_range(WS['Date Assigned'])

for tbl, name, caption in [(WS, 'PCE', 'Plan Checks from {} to {}'.format(WSmn, WSmx))]:
    worksheet = wrkbk.add_worksheet(name)
    
    lyr = tbl.copy()
    
    lyr = normalize_types(lyr)
    
    header = np.array([[lyr.index.name] + list(lyr)])
    data = lyr.reset_index().to_numpy()
    if data.shape[1] != header.shape[1]:        
        dif = data.shape[1] - header.shape[1]
        fix_head = [lyr.index.name] + list(lyr)
        for r in range(dif):
            fix_head = [' '] + fix_head
        header = np.array([fix_head])
    table = np.vstack((header,data))
    numRows,numColumns = table.shape
    
    cols = [{'header':h} for h in header[0]]
    
    rowoffset,coloffset = 1,0
    
    if len(caption) > 0:
        cell_format = wrkbk.add_format()
        cell_format.set_bold() 
        worksheet.merge_range('A1:B1', caption, cell_format)
        worksheet.write('D1', 'updated on {}'.format(datetime.now().strftime('%m-%d-%Y')))
        rowoffset += 1
    
    length,width = data.shape
    table_range = rng_create(0+coloffset, 0+rowoffset, width-1+coloffset, length+rowoffset)
    
    worksheet.add_table(table_range, {'data': data,
                                      'style': 'Table Style Light 1',
                                       'columns': cols
                                 })
    for e,c in enumerate(get_col_widths(lyr)):
        e = e+coloffset
        cw = colnum_string(e)
        cw = cw + ':' + cw
        if c > 30:
            c = 30
        worksheet.set_column(cw, c+1)
    
wrkbk.close()